In [0]:
%pip install pydicom==3.0.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.4 MB ? eta -:--:--
   ━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.8/2.4 MB 22.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.9 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import logging
logger = logging.getLogger("zipdcm")
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(filename)s:%(lineno)s - %(levelname)s - %(message)s')
# Create a handler (e.g., StreamHandler for console output)
handler = logging.StreamHandler()
handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(handler)

### How many zip files 
do we have on `/Volumes/hls_radiology/tcia/downloads/tciaDownload`

In [0]:
# how many zips do we have?
df = (
    spark.read.format("binaryFile")
        .option("pathGlobFilter", "*.zip")
        .option("recursiveFileLookup", "true")
        .load("/Volumes/hls_radiology/tcia/downloads/tciaDownload").drop('content')
    )
df.count()

1416

In [0]:
from zip_dcm_ds import ZipDCMDataSource

import time
start_time = time.time()
total_worker_cores = 16 


# Add our custom Python DataSource for DICMO files storaged in a Zip Archive
spark.dataSource.register(ZipDCMDataSource)

def test_scale(spark, numPartitions=64):
    df = (
        spark.read
          .option('numPartitions',f'{numPartitions}')
          .format("zipdcm")
          .load("/Volumes/hls_radiology/tcia/downloads/tciaDownload") #1416 zip files
    )
    df.write.format('delta').mode('overwrite').saveAsTable("douglas_moore.pixels.zipdcm_test")

test_scale(spark)


end_time = time.time()
operation_time = end_time - start_time
logger.info(f"Operation time: {operation_time} seconds and {operation_time * total_worker_cores} core-seconds")

2025-05-23 17:50:19,325 - command-2588019255038221-3081177968:25 - INFO - Operation time: 217.42620420455933 seconds and 3478.819267272949 core-seconds


## Summary stats

In [0]:
%sql
select count(distinct zipfile) zipfiles, count(distinct zipfile, dcmfile) dcmfiles, count(1) records
from douglas_moore.pixels.zipdcm_test

zipfiles,dcmfiles,records
1416,107158,107158


In [0]:
%sql
select zipfile,count(distinct dcmfile) 
from douglas_moore.pixels.zipdcm_test
group by zipfile 
order by 2 desc

zipfile,count(DISTINCT dcmfile)
/Volumes/hls_radiology/tcia/downloads/tciaDownload/1.3.6.1.4.1.14519.5.2.1.8700.9920.236081364348443124124483438380.zip,2115
/Volumes/hls_radiology/tcia/downloads/tciaDownload/1.3.6.1.4.1.14519.5.2.1.8700.9920.653526830498876067334206778015.zip,2115
/Volumes/hls_radiology/tcia/downloads/tciaDownload/1.3.6.1.4.1.14519.5.2.1.8700.9920.220594080380513351538894535345.zip,2115
/Volumes/hls_radiology/tcia/downloads/tciaDownload/3.4.585.0.0.8284097.9.501.1675956140590704039.zip,2115
/Volumes/hls_radiology/tcia/downloads/tciaDownload/1.5.168.0.1.5553498.1.707.5757025447751332003.zip,2115
/Volumes/hls_radiology/tcia/downloads/tciaDownload/3.3.308.1.2.2873245.5.858.2631549223322125879.zip,2115
/Volumes/hls_radiology/tcia/downloads/tciaDownload/3.3.193.0.1.9916426.8.134.1218337709620319560.zip,2115
/Volumes/hls_radiology/tcia/downloads/tciaDownload/1.3.6.1.4.1.14519.5.2.1.8700.9920.146883340873245283101117847180.zip,2115
/Volumes/hls_radiology/tcia/downloads/tciaDownload/3.1.755.1.3.3756213.9.632.2823904128812494525.zip,2025
/Volumes/hls_radiology/tcia/downloads/tciaDownload/1.3.6.1.4.1.14519.5.2.1.8700.9920.215964082035969762468110236185.zip,2025


Databricks visualization. Run in Databricks to view.

## Sample data records

In [0]:
%sql
select * 
from douglas_moore.pixels.zipdcm_test
limit 100;

rowid zipfile dcmfile meta 331 /Volumes/hls_radiology/tcia/downloads/tciaDownload/1.3.6.1.4.1.14519.5.2.1.8700.9920.180074380594221583725656478944.zip 1-1.dcm {'00080005': {'vr': 'CS', 'Value': ['ISO_IR 100']}, '00080008': {'vr': 'CS', 'Value': ['ORIGINAL', 'PRIMARY', '']}, '00080016': {'vr': 'UI', 'Value': ['1.2.840.10008.5.1.4.1.1.1.1']}, '00080018': {'vr': 'UI', 'Value': ['1.3.6.1.4.1.14519.5.2.1.8700.9920.512812470930484087715809150930']}, '00080020': {'vr': 'DA', 'Value': ['20330416']}, '00080021': {'vr': 'DA', 'Value': ['20330416']}, '00080022': {'vr': 'DA', 'Value': ['20330416']}, '00080023': {'vr': 'DA', 'Value': ['20330416']}, '00080024': {'vr': 'DA', 'Value': ['20330416']}, '00080025': {'vr': 'DA', 'Value': ['20330416']}, '0008002A': {'vr': 'DT', 'Value': ['20330416140502']}, '00080030': {'vr': 'TM', 'Value': ['140502']}, '00080032': {'vr': 'TM', 'Value': ['140639']}, '00080033': {'vr': 'TM', 'Value': ['140502']}, '00080050': {'vr': 'SH'}, '00080060': {'vr': 'CS', 'Value': ['DX']}, '00080070': {'vr': 'LO', 'Value': ['GE MEDICAL SYSTEMS']}, '00080090': {'vr': 'PN'}, '00081030': {'vr': 'LO', 'Value': ['CHEST 2 VIEW']}, '0008103E': {'vr': 'LO', 'Value': ['CHEST 2 VIEW']}, '00081090': {'vr': 'LO', 'Value': ['Revolution XQi ADS_28.4']}, '00081155': {'vr': 'UI', 'Value': ['1.3.6.1.4.1.14519.5.2.1.8700.9920.327237660668073937712027718550']}, '00082218': {'vr': 'SQ', 'Value': [{'00080100': {'vr': 'SH', 'Value': ['T-D3000']}, '00080102': {'vr': 'SH', 'Value': ['SNM3']}, '00080104': {'vr': 'LO', 'Value': ['Chest']}}]}, '00100010': {'vr': 'PN', 'Value': [{'Alphabetic': 'MIDI_1_1_176'}]}, '00100020': {'vr': 'LO', 'Value': ['MIDI_1_1_176']}, '00100030': {'vr': 'DA'}, '00100040': {'vr': 'CS', 'Value': ['F']}, '00101010': {'vr': 'AS', 'Value': ['026Y']}, '00101020': {'vr': 'DS', 'Value': [1.69]}, '00101030': {'vr': 'DS', 'Value': [82.946]}, '001021D0': {'vr': 'DA', 'Value': ['20330318']}, '00120062': {'vr': 'CS', 'Value': ['YES']}, '00120063': {'vr': 'LO', 'Value': ['Per DICOM PS 3.15 AnnexE. Details in 0012,0064']}, '00120064': {'vr': 'SQ', 'Value': [{'00080100': {'vr': 'SH', 'Value': ['113100']}, '00080102': {'vr': 'SH', 'Value': ['DCM']}, '00080104': {'vr': 'LO', 'Value': ['Basic Application Confidentiality Profile']}}, {'00080100': {'vr': 'SH', 'Value': ['113101']}, '00080102': {'vr': 'SH', 'Value': ['DCM']}, '00080104': {'vr': 'LO', 'Value': ['Clean Pixel Data Option']}}, {'00080100': {'vr': 'SH', 'Value': ['113104']}, '00080102': {'vr': 'SH', 'Value': ['DCM']}, '00080104': {'vr': 'LO', 'Value': ['Clean Structured Content Option']}}, {'00080100': {'vr': 'SH', 'Value': ['113105']}, '00080102': {'vr': 'SH', 'Value': ['DCM']}, '00080104': {'vr': 'LO', 'Value': ['Clean Descriptors Option']}}, {'00080100': {'vr': 'SH', 'Value': ['113107']}, '00080102': {'vr': 'SH', 'Value': ['DCM']}, '00080104': {'vr': 'LO', 'Value': ['Retain Longitudinal Temporal Information Modified Dates Option']}}, {'00080100': {'vr': 'SH', 'Value': ['113108']}, '00080102': {'vr': 'SH', 'Value': ['DCM']}, '00080104': {'vr': 'LO', 'Value': ['Retain Patient Characteristics Option']}}, {'00080100': {'vr': 'SH', 'Value': ['113109']}, '00080102': {'vr': 'SH', 'Value': ['DCM']}, '00080104': {'vr': 'LO', 'Value': ['Retain Device Identity Option']}}, {'00080100': {'vr': 'SH', 'Value': ['113111']}, '00080102': {'vr': 'SH', 'Value': ['DCM']}, '00080104': {'vr': 'LO', 'Value': ['Retain Safe Private Option']}}]}, '00130010': {'vr': 'LO', 'Value': ['CTP']}, '00131010': {'vr': 'LO', 'Value': ['MIDI-B-Curated-Validation']}, '00131013': {'vr': 'LO', 'Value': ['29059806']}, '00131015': {'vr': 'LO', 'Value': ['2']}, '00180015': {'vr': 'CS', 'Value': ['CHEST']}, '00180060': {'vr': 'DS', 'Value': [125.0]}, '00181020': {'vr': 'LO', 'Value': ['Ads Application Package VERSION ADS_28.4']}, '00181030': {'vr': 'LO', 'Value': ['Performed']}, '00181110': {'vr': 'DS', 'Value': [1800.0]}, '00181111': {'vr': 'DS', 'Value': [1750.0]}, '00181147': {'vr': 'CS', 'Value': ['RECTANGLE']}, '0018114